### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).





In [24]:
# calculating psat water and psat 1,4 dioxide
import math
import numpy as np
import torch as t

from torch.autograd import Variable
a = Variable(t.tensor([2.,1.]), requires_grad=True)

b=torch.tensor([[8.07131,1730.63,233.426],[7.43155,1554.679,240.337]])
#psat=torch.tensor([ ])
lm=[]
x1=torch.tensor([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
pexp=torch.tensor([28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])
loss=0
    
for i in range(2):
    l=b[i][1]/(20+b[i][2])
    m=b[i][0]-l
    d=[10**m]
    lm.append(d)

psat=torch.tensor(lm)
start_step = 0.01 

for j in range(1):     # keep searching while gradient norm is larger than eps
    loss=0
    for i in range(len(x1)):
        x2=1-x1[i]
        l1=a[1]*x2/(a[0]*x1[i]+a[1]*x2)
        l1woexp=a[0]*(l1**2)    
        l1wexp=torch.exp(l1woexp)
        l1full=x1[i]*l1wexp*psat[0]
        r1=a[0]*x1[i]/(a[0]*x1[i]+a[1]*x2)
        r1woexp=a[1]*(l1**2)
        r1wexp=torch.exp(r1woexp)
        r1full=x2*r1wexp*psat[1]  
        pcal=l1full+r1full           
        loss=loss+(pcal-pexp[i])**2

    obj = lambda a:loss
    
    print(obj(a))
    

    loss.backward()
    print(a.grad)

    
    
    with t.no_grad():
        a -= start_step * a.grad
        
        # need to clear the gradient at every step, or otherwise it will accumulate...
        a.grad.zero_()
        

print(a.data.numpy())
print(loss.data.numpy())  

tensor([3817.4685], grad_fn=<AddBackward0>)
tensor([ -109.3798, 10140.3730])
[   3.093798 -100.403725]
[3817.4685]


In [ ]:
 #!pip install bayesian-optimization
    
from bayes_opt import BayesianOptimization
def fun(x0,x1):
    return -((4-(2.1*x0**2)+((x0**4)/3))*x0**2+(x0*x1)+(-4+(4*x1**2))*x1**2)
    from bayes_opt import BayesianOptimization
limits= {'x0': (-3,3), 'x1': (-2,2)}

optimizer= BayesianOptimization(f=fun, pbounds=limits, random_state=1)
optimizer.maximize(init_points=2,n_iter=50)
print(optimizer.max)

RuntimeError: Trying to backward through the graph a second time (or directly access saved variables after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved variables after calling backward.